In [1]:
from Lib.RPS_game import play, mrugesh, abbey, quincy, kris, human, random_player
from Lib.RPS_encoding import RPSpair_encode, RPSpair_decode, RPSpair_encode_series
from Lib.Telepathic_player import telepath
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import Dense, LSTM, Embedding
import pandas as pd
from os import listdir
from os.path import isfile, join
import random
import itertools
import tensorflow_probability as tfp
from tensorflow_probability import distributions as tfd

In [2]:

path = "Games/"
all_game_paths = [join(path, f) for f in listdir(path) if isfile(join(path, f))]
games_concat = pd.DataFrame(columns = ["Player","Opponent","Result","Winning_Play"])
games = []
for game_path in all_game_paths:
    #print(game_path)
    game_data = pd.read_csv(game_path, names = ["Player","Opponent","Result","Winning_Play"])
    games.append(game_data)
    games_concat = games_concat.append(game_data)
    
#print(len(game_data.index)) 
#game_data.head()  
print(len(games_concat.index))
games_concat.head()

100000


,Player,Opponent,Result,Winning_Play
0,S,S,0,R
1,R,P,0,S
2,R,R,0,P
3,P,R,1,P
4,R,P,0,S


In [3]:
temp = games_concat.drop("Result", axis = 1)
temp.index = [x for x in range(temp.index.size)]
tempInput = pd.Series([temp.loc[x,"Player"] + temp.loc[x,"Opponent"] for x in range(temp['Player'].size)])
tempTarget = pd.Series([temp.loc[x,"Winning_Play"] + temp.loc[x,"Opponent"] for x in range(temp['Player'].size)])

tempInput = RPSpair_encode_series(tempInput)
tempTarget = RPSpair_encode_series(tempTarget)


In [4]:
input_data = tf.data.Dataset.from_tensor_slices(tempInput)
target_data = tf.data.Dataset.from_tensor_slices(tempTarget)

seq_length = 100
#split into sequences
input_sequences = input_data
target_sequences = target_data
print(input_sequences)

#remove last element from all input sequences
def remove_last(chunk):
    return chunk[:-1]
inputs = input_sequences.map(remove_last)

#remove first element from all target sequences
def remove_first(chunk):
    return chunk[1:]
targets = target_sequences.map(remove_first)
    
full_dataset = tf.data.Dataset.zip((inputs, targets)).shuffle(10000)

dataset_size = len(full_dataset)
train_size = int(0.7*dataset_size)
test_size = int(0.15*dataset_size)
val_size = int(0.15*dataset_size)

train_dataset = full_dataset.take(train_size)
test_dataset = full_dataset.skip(train_size)
val_dataset = test_dataset.take(test_size)
test_dataset = test_dataset.skip(test_size)

print("Total sequence count: " + str(dataset_size))
print("Training set: " + str(len(train_dataset)))
print("Test set: " + str(len(test_dataset)))
print("Validation set: " + str(len(val_dataset)))
for element in train_dataset.take(1):
    print(element)


    

<TensorSliceDataset shapes: (), types: tf.int64>


ValueError: in user code:

    <ipython-input-4-db0520606672>:12 remove_last  *
        return chunk[:-1]
    c:\users\owner\appdata\local\programs\python\python38\lib\site-packages\tensorflow\python\util\dispatch.py:201 wrapper
        return target(*args, **kwargs)
    c:\users\owner\appdata\local\programs\python\python38\lib\site-packages\tensorflow\python\ops\array_ops.py:1013 _slice_helper
        return strided_slice(
    c:\users\owner\appdata\local\programs\python\python38\lib\site-packages\tensorflow\python\util\dispatch.py:201 wrapper
        return target(*args, **kwargs)
    c:\users\owner\appdata\local\programs\python\python38\lib\site-packages\tensorflow\python\ops\array_ops.py:1186 strided_slice
        op = gen_array_ops.strided_slice(
    c:\users\owner\appdata\local\programs\python\python38\lib\site-packages\tensorflow\python\ops\gen_array_ops.py:10347 strided_slice
        _, _, _op, _outputs = _op_def_library._apply_op_helper(
    c:\users\owner\appdata\local\progra

In [8]:
import random


class RandomPredictor():

    @staticmethod
    def predict():
        return random.choice(['R','P','S'])
    
class MarkovChain():

    def __init__(self, order, decay=1.0):
        self.decay = decay
        self.matrix = self.create_matrix(order)

    @staticmethod
    def create_matrix(order):

        def create_keys(order):            

            keys = ['R', 'P', 'S']

            for i in range(order):
                key_len = len(keys)
                for i in itertools.product(keys, ''.join(keys)):
                    keys.append(''.join(i))
                keys = keys[key_len:]

            return keys

        keys = create_keys(order)

        matrix = {}
        for key in keys:
            matrix[key] = {'R': {'prob' : 1 / 3,
                                 'n_obs' : 0
                                },
                           'P': {'prob' : 1 / 3,
                                 'n_obs' : 0
                                },
                           'S': {'prob' : 1 / 3,
                                 'n_obs' : 0
                                }
                          }
        #print(matrix)
        return matrix

    def update_matrix(self, pair, input):
        
        for i in self.matrix[pair]:
            self.matrix[pair][i]['n_obs'] = self.decay * self.matrix[pair][i]['n_obs']

        self.matrix[pair][input]['n_obs'] = self.matrix[pair][input]['n_obs'] + 1
        
        n_total = 0
        for i in self.matrix[pair]:
            n_total += self.matrix[pair][i]['n_obs']
            
        for i in self.matrix[pair]:
            self.matrix[pair][i]['prob'] = self.matrix[pair][i]['n_obs'] / n_total            

    def predict(self, pair):

        probs = self.matrix[pair]

        if max(probs.values()) == min(probs.values()):
            return random.choice(['R', 'P', 'S'])
        else:
            return max([(i[1], i[0]) for i in probs.items()])[1]        
    
    
markov_model = MarkovChain(3, 0.9)


In [7]:
model = RandomPredictor()
output = model.predict()

In [ ]:
beat = {'R': 'P', 'P': 'S', 'S': 'R'}


# the first round
if input == '':

    random_predictor = RandomPredictor()
    markov_model = MarkovChain(1, 0.9)

    pair_diff2 = ''
    pair_diff1 = ''


# further rounds
else:
    pair_diff2 = pair_diff1
    pair_diff1 = output + input


if pair_diff2 != '':
    markov_model.update_matrix(pair_diff2, input)
    output = beat[markov_model.predict(pair_diff1)]

else:
    output = random_predictor.predict()

## Tensorflow ver

In [2]:
# A simple weather model.

# Represent a cold day with 0 and a hot day with 1.
# Suppose the first day of a sequence has a 0.8 chance of being cold.
# We can model this using the categorical distribution:

initial_distribution = tfd.Categorical(probs=[0.8, 0.2])

# Suppose a cold day has a 30% chance of being followed by a hot day
# and a hot day has a 20% chance of being followed by a cold day.
# We can model this as:

transition_distribution = tfd.Categorical(probs=[[0.7, 0.3],
                                                 [0.2, 0.8]])

# Suppose additionally that on each day the temperature is
# normally distributed with mean 0 and standard deviation 5 on
# a cold day and mean 15 and standard deviation 10 on a hot day.
# We can model this with:

observation_distribution = tfd.Normal(loc=[0., 15.], scale=[5., 10.])

# We can combine these distributions into a single week long
# hidden Markov model with:

model = tfd.HiddenMarkovModel(
    initial_distribution=initial_distribution,
    transition_distribution=transition_distribution,
    observation_distribution=observation_distribution,
    num_steps=7)

# The expected temperatures for each day are given by:

model.mean()  # shape [7], elements approach 9.0

# The log pdf of a week of temperature 0 is:

model.log_prob(tf.zeros(shape=[7]))

<tf.Tensor: shape=(), dtype=float32, numpy=-19.855635>

In [33]:
#print(tfd.Normal(loc=[0., 15.], scale=[5., 10.]))

tfp.distributions.Normal("Normal", batch_shape=[], event_shape=[], dtype=float32)


In [47]:
from Lib.RPS_encoding import RPS_encode
def create_states(count):            
    keys = ['R', 'P', 'S']
    states = []
    for i in itertools.product(keys, repeat=count):
        states.append(''.join(i))
    return states
order = 3
states = create_states(order)

#RPS model
#initial state probabilities first 3 RPS have an even chance of occuring distribution
init_probs = [1/len(states) for state in states] 
initial_distribution = tfd.Categorical(probs=init_probs)
print(init_probs)

#some transitions can never occur so transition prob init to 0 
#from each state can transition to x (order) possible states
transition_probs_mask = [[next_state[:-1] == current_state[1:] for next_state in states] for current_state in states]
transition_probs = [[1/order if next_state[:-1] == current_state[1:] else 0 for next_state in states] for current_state in states]
print(transition_probs)
transition_distribution = tfd.Categorical(probs = transition_probs)

# For each outcome state the output is a definite R, P or S
#Get correct output for each state in order
default = [0, 0, 0]
observation_probs = [[1, 0, 0] if state[-1] == 'R' else [0, 1, 0] if state[-1] == 'P' else [0, 0, 1] for state in states]
observation_distribution = tfd.Categorical(probs = observation_probs)
print(states)
print(observation_probs)
print(observation_distribution)

[0.037037037037037035, 0.037037037037037035, 0.037037037037037035, 0.037037037037037035, 0.037037037037037035, 0.037037037037037035, 0.037037037037037035, 0.037037037037037035, 0.037037037037037035, 0.037037037037037035, 0.037037037037037035, 0.037037037037037035, 0.037037037037037035, 0.037037037037037035, 0.037037037037037035, 0.037037037037037035, 0.037037037037037035, 0.037037037037037035, 0.037037037037037035, 0.037037037037037035, 0.037037037037037035, 0.037037037037037035, 0.037037037037037035, 0.037037037037037035, 0.037037037037037035, 0.037037037037037035, 0.037037037037037035]
[[0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0

In [48]:
model = tfd.HiddenMarkovModel(
    initial_distribution=initial_distribution,
    transition_distribution=transition_distribution,
    observation_distribution=observation_distribution,
    num_steps=7)

In [55]:
# The expected ouputs for each step are given by:
from Lib.RPS_encoding import RPS_decode
predictions = model.sample()  # shape [7]
print(predictions)
for prediction in predictions:
    print(RPS_decode(prediction))
# The log pdf of a week of temperature 0 is:

tf.Tensor([1 2 0 0 1 0 2], shape=(7,), dtype=int32)
P
S
R
R
P
R
S


AttributeError: 'HiddenMarkovModel' object has no attribute 'fit'